In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install timm --no-index --find-links=file:///kaggle/input/timm-package/

In [ ]:
!pip install albumentations --no-index --find-links=file:///kaggle/input/albumentationspackage/

In [ ]:
!ls /kaggle/input

In [ ]:
import sys
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import Parameter
import os
import cv2
import timm

In [ ]:
import albumentations as A

In [ ]:
def gem(x, p=3, eps=1e-5):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):

    def __init__(self, p=3, eps=1e-5):
        super(GeM, self).__init__()
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)

    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(
            self.eps) + ')'

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()

        # self.mean_tensor=torch.from_numpy(cfg.DATA.PIXEL_MEAN ).float().cuda()
        # self.std_val_tensor = torch.from_numpy(cfg.DATA.PIXEL_STD).float().cuda()
        # self.model = EfficientNet.from_pretrained(model_name='efficientnet-b0')
        # self.model = timm.create_model('mobilenetv2_110d', pretrained=True)

        # self.model = timm.create_model('mobilenetv2_110d', pretrained=True)
        self.model = timm.create_model('seresnext50_32x4d', pretrained=False)

        self._avg_pooling = nn.AdaptiveAvgPool2d(1)

        self.dropout=nn.Dropout(0.5)

        self._fc = nn.Linear(2048 , num_classes, bias=True)

    def forward(self, inputs):

        #do preprocess

        input_iid = inputs
        input_iid=input_iid/255.
        bs = input_iid.size(0)
        # Convolution layers
        x = self.model.forward_features(input_iid)
        fm = self._avg_pooling(x)
        fm = fm.view(bs, -1)
        feature=self.dropout(fm)


        x = self._fc(feature)


        return x

In [ ]:
class DatasetTest():
    def __init__(self, test_data_dir):
        self.ds = self.get_list(test_data_dir)

        self.root_dir = test_data_dir
        
        self.val_trans=A.Compose([A.HorizontalFlip(p=0.5),
                                    A.VerticalFlip(p=0.5),
                                    A.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2, hue=0.00, always_apply=False, p=1.0),
                                    A.RandomCrop(height= 600, width = 600,always_apply=True, p=1.0)]
                                )

        
    def get_list(self, dir):
        pic_list = os.listdir(dir)
        
        return pic_list

    def __len__(self):

        return len(self.ds)
     
    def preprocess_func(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        image1=self.val_trans(image=image)['image']
        image2=self.val_trans(image=image)['image']
        image3=self.val_trans(image=image)['image']
        image4=self.val_trans(image=image)['image']
        image5=self.val_trans(image=image)['image']
        image6=self.val_trans(image=image)['image']
        image7=self.val_trans(image=image)['image']
        image8=self.val_trans(image=image)['image']
        
        image_batch = np.stack([image1,
                                image2,
                                image3,
                                image4,
                                image5,
                                image6,
                                image7,
                                image8
                               ])

        image_batch = np.transpose(image_batch, axes=[0, 3, 1, 2])
        return image, image_batch

    def __getitem__(self, item):
        fname = self.ds[item]

        image_path = os.path.join(self.root_dir, fname)
        image = cv2.imread(image_path, -1)
        image,float_image = self.preprocess_func(image)

        return fname, image,float_image

In [ ]:
# class DatasetTest():
#     def __init__(self, test_data_dir):
#         self.ds = self.get_list(test_data_dir)

#         self.root_dir = test_data_dir

#     def get_list(self, dir):
#         pic_list = os.listdir(dir)

#         return pic_list

#     def __len__(self):

#         return len(self.ds)

#     def preprocess_func(self, image):
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image=cv2.resize(image,(640,640))

#         image_90=np.rot90(image,1)
#         image_180 = np.rot90(image, 2)
#         image_270 = np.rot90(image, 3)

#         image_fliplr = np.fliplr(image)
#         image_fliplr_90 = np.rot90(image, 1)
#         image_fliplr_180 = np.rot90(image, 2)
#         image_fliplr_270 = np.rot90(image, 3)

#         image_batch = np.stack([image,image_90,image_180,image_270,
#                                 image_fliplr,image_fliplr_90,image_fliplr_180,image_fliplr_270])

#         image_batch = np.transpose(image_batch, axes=[0, 3, 1, 2])
#         return image, image_batch

#     def __getitem__(self, item):
#         fname = self.ds[item]

#         image_path = os.path.join(self.root_dir, fname)
#         image = cv2.imread(image_path, -1)
#         image,float_image = self.preprocess_func(image)

#         return fname, image,float_image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kaggle_root = '/kaggle/input'
model_dir = os.path.join(kaggle_root, 'cassva-models-se50-640')
weights = [os.path.join(model_dir, f) for f in os.listdir(model_dir)]

test_datadir= os.path.join(kaggle_root, 'cassava-leaf-disease-classification/test_images')

dataiter=DatasetTest(test_datadir)


def predict_with_model(model,weights):
    merge_res_dict = {}    
    for j, weight in enumerate(weights):
        model.load_state_dict(torch.load(weight, map_location=device), strict=False)
        ### load your weights
        model.eval()

        cur_result=pd.DataFrame(columns=['image_id','label'])
        
        len_data = len(dataiter)

        image_ids=[]
        precictions=[]
        for i in range(len(dataiter)):
            print('weight {}: data {}/{}'.format(j, i+1, len_data))
            fname,_,float_image=dataiter.__getitem__(i)

            input=torch.from_numpy(float_image).to(device).float()
            with torch.no_grad():
                output=model(input)

                output=torch.nn.functional.softmax(output,dim=-1)
                output=output.cpu().numpy()

                output=np.mean(output,axis=0)

            image_ids.append(fname)

            label=np.argmax(output)

            precictions.append(label)
            if fname not in merge_res_dict:
                merge_res_dict[fname] = output
            else:
                merge_res_dict[fname] += output

#         cur_result['image_id']=image_ids
#         cur_result['label'] = precictions
        
#         cur_result.to_csv('{}.csv'.format(weight.split('/')[-1]),index=False)
#         cur_result.to_csv('submission.csv',index=False)
        
    merge_result = pd.DataFrame(columns=['image_id','label'])
    precictions = []
    for fname in merge_res_dict:
        label = np.argmax(merge_res_dict[fname])
        precictions.append(label)
    merge_result['image_id'] = image_ids
    merge_result['label'] = precictions
#     print(merge_res_dict)
#     print(merge_result)
    merge_result.to_csv('submission.csv',index=False)
    

model=Net().to(device)

predict_with_model(model,weights)